# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [1]:
import gradio as gr

In [2]:
# Import required libraries
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [3]:
# Load environment variables from .env file
load_dotenv(override=True)

# API keys from environment
openai_api_key = os.getenv('OPENAI_API_KEY')
ollama_base_url = os.getenv('OLLAMA_BASE_URL')
ollama_api_key = os.getenv('OLLAMA_API_KEY')
ollama_model = os.getenv('OLLAMA_MODEL', 'qwen3-coder:480b-cloud')

# Verify API keys
if openai_api_key:
    print(f"OpenAI API Key loaded: {openai_api_key[:8]}...")
else:
    print("OpenAI API Key not set")

if ollama_base_url:
    print(f"Ollama configured at: {ollama_base_url}")
    print(f"Ollama Model is : {ollama_model}")
    

OpenAI API Key loaded: sk-proj-...
Ollama configured at: http://192.168.80.200:11434
Ollama Model is : qwen3-coder:480b-cloud


In [4]:
# Initialize API clients
openai_client = OpenAI(api_key=openai_api_key)

# Initialize Ollama client (uses OpenAI-compatible API)
ollama_client = OpenAI(
    base_url=f"{ollama_base_url}/v1",
    api_key=ollama_api_key
)

print("All clients initialized successfully")

All clients initialized successfully


In [26]:
system_message = """
Eres un asistente util para una aerolinea llamada FlightAI.
Da respuestas breves y corteses, de no mas de una oración.
Se siempre preciso. Si no sabes la respuesta, dilo.
"""

In [27]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    stream = ollama_client.chat.completions.create(
        model=ollama_model,
        messages=messages,
        stream = True
    )

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [28]:
gr.close_all()
gr.ChatInterface(fn=chat, type="messages").launch(share=True)

Closing server running on port: 7860
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://1aa54695b78e592cc5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [29]:
# Let's start by making a useful function

ticket_prices = {"londres": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool called for city {destination_city}")
    price = ticket_prices.get(destination_city.lower(), "Unknown ticket price")
    return f"The price of a ticket to {destination_city} is {price}"


In [30]:
get_ticket_price("londres")

Tool called for city londres


'The price of a ticket to londres is $799'

In [31]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [32]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

In [33]:
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': 'Get the price of a return ticket to the destination city.',
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}},
    'required': ['destination_city'],
    'additionalProperties': False}}}]

## Testing Tool Support in Ollama Models

Let's test which Ollama models support function calling properly.

In [34]:
# List of Ollama models to test
ollama_models_to_test = [
    "deepseek-v3.1:671b-cloud",
    "gpt-oss:20b-cloud",
    "gpt-oss:120b-cloud",
    "kimi-k2:1t-cloud",
    "qwen3-coder:480b-cloud",
    "glm-4.6:cloud",
    "minimax-m2:cloud"
]

def test_model_tools(model_name):
    """Test if a model supports tool calling"""
    try:
        test_messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "What's the price of a ticket to Paris?"}
        ]
        
        response = ollama_client.chat.completions.create(
            model=model_name,
            messages=test_messages,
            tools=tools,
            max_tokens=100
        )
        
        finish_reason = response.choices[0].finish_reason
        has_tool_calls = hasattr(response.choices[0].message, 'tool_calls') and response.choices[0].message.tool_calls
        
        return {
            "model": model_name,
            "finish_reason": finish_reason,
            "supports_tools": finish_reason == "tool_calls",
            "has_tool_calls": has_tool_calls,
            "response": response.choices[0].message.content if not has_tool_calls else "Called tool"
        }
    except Exception as e:
        return {
            "model": model_name,
            "error": str(e)
        }

In [35]:
# Test all models
print("Testing tool support for Ollama models...\n")
print("=" * 80)

results = []
for model in ollama_models_to_test:
    print(f"\nTesting: {model}")
    result = test_model_tools(model)
    results.append(result)
    
    if "error" in result:
        print(f"  Error: {result['error'][:100]}")
    elif result.get("supports_tools"):
        print(f"  SUPPORTS TOOLS! finish_reason={result['finish_reason']}")
    else:
        print(f"  No tool support. finish_reason={result['finish_reason']}")
        print(f"  Response: {result.get('response', 'N/A')[:80]}")

print("\n" + "=" * 80)
print("\nSUMMARY - Models with Tool Support:")
for r in results:
    if r.get("supports_tools"):
        print(f"  {r['model']}")
        
print("\nModels WITHOUT Tool Support:")
for r in results:
    if not r.get("supports_tools") and "error" not in r:
        print(f"   {r['model']}")

Testing tool support for Ollama models...


Testing: deepseek-v3.1:671b-cloud
  SUPPORTS TOOLS! finish_reason=tool_calls

Testing: gpt-oss:20b-cloud
  SUPPORTS TOOLS! finish_reason=tool_calls

Testing: gpt-oss:120b-cloud
  SUPPORTS TOOLS! finish_reason=tool_calls

Testing: kimi-k2:1t-cloud
  Error: Error code: 404 - {'error': {'message': "model 'kimi-k2:1t-cloud' not found", 'type': 'api_error', '

Testing: qwen3-coder:480b-cloud
  SUPPORTS TOOLS! finish_reason=tool_calls

Testing: glm-4.6:cloud
  Error: Error code: 404 - {'error': {'message': "model 'glm-4.6:cloud' not found", 'type': 'api_error', 'par

Testing: minimax-m2:cloud
  Error: Error code: 404 - {'error': {'message': "model 'minimax-m2:cloud' not found", 'type': 'api_error', '


SUMMARY - Models with Tool Support:
  deepseek-v3.1:671b-cloud
  gpt-oss:20b-cloud
  gpt-oss:120b-cloud
  qwen3-coder:480b-cloud

Models WITHOUT Tool Support:


## Getting Ollama to use our Tool

Based on testing, **qwen3-coder:480b-cloud** has the best Tool calling support.

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [41]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    # Use qwen3-coder:480b-cloud - best Tool support from testing
    response = ollama_client.chat.completions.create(
        model=ollama_model,
        messages=messages,
        tools=tools
    )

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = ollama_client.chat.completions.create(
            model=ollama_model,
            messages=messages
        )
    
    return response.choices[0].message.content

In [42]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name == "get_ticket_price":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price_details = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": price_details,
            "tool_call_id": tool_call.id
        }
    return response

In [15]:
gr.close_all()
gr.ChatInterface(fn=chat, type="messages").launch(share=True)

Closing server running on port: 7860
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://a039c1b9fa74ae8e40.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Tool called for city London


## Let's make a couple of improvements

Handling multiple tool calls in 1 response

Handling multiple tool calls 1 after another

In [43]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    response = ollama_client.chat.completions.create(
        model=ollama_model,
        messages=messages,
        tools=tools
    )
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = ollama_client.chat.completions.create(
            model=ollama_model,
            messages=messages,
            tools=tools
        )
    
    return response.choices[0].message.content

In [44]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            price_details = get_ticket_price(city)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
    return responses

In [45]:
gr.close_all()
gr.ChatInterface(fn=chat, type="messages").launch(share=True)

Closing server running on port: 7860
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://e03e324a88c8916a24.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Tool called for city Londres
Tool called for city tokio
Tool called for city tokyo
